I'm using Gemini model with LLamaIndex to make an AI customer service agent. It can respond to queries and take orders. The menu and other restaurant info is in the system prompt. 

There is pydantic object named Response with a key response and a key status. response key holds the llm response, status key holds the status of system. If customer asks a general query, it is normal, if they are trying to order somethihng it is order, if they are done with order, it is done. 

When customer starts ordering, kick in the order module that keeps track of all the items added to cart. Whenever the customers adds something, ask if they want something else, if they say no or something that suggests that they are done, mark the order complete, tell them the summary and ask for their name and phone number, end the session and return the order in json. 

This should follow a general order taking conversation, asks for customization or other details if they are provided in menu such as chicken optiins and spice levels, also ask for other instructions.

In [ ]:
GOOGLE_API_KEY = "Add api key"
import os
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [2]:
with open("./restaurant_details.txt", "r") as f:
    detail = f.read()

In [3]:
SYSTEM_PROMPT = f"""You are a customer service agent at a restaurant. Your job is to answer customer questions or take order if they want to place an order. Menu and other details of restuarant are as follows.
When someone asks about menu. Give them an overview, don't go in details. Go in details only if asked.
{detail}

Respond according to the given JSON schema.
"""


In [4]:
# resp = llm.complete(f"{SYSTEM_PROMPT}\nOkay, I want a loaded fries" )
# print(resp)

In [5]:
# import json
# response_json = json.loads(resp.text)
# response = response_json["response"]
# order = response_json["order"]

In [6]:
import firebase_admin
from firebase_admin import credentials, firestore
import random
import datetime

cred = credentials.Certificate("ezrai-4ccbe-firebase-adminsdk-fbsvc-ef41610e60.json")
firebase_admin.initialize_app(cred)
db = firestore.client()

def push_order(order):
    """Pushes multiple dummy orders to Firestore."""
    orders_ref = db.collection("orders")
    ct = orders_ref.count()
    count_query = orders_ref.count()
    count_result = count_query.get()

    # Extract count
    ct = count_result[0][0].value
    orders_ref.document(str(ct+1)).set({"items":order})

    print(f"✅ Order {order} added.")


In [7]:
from llama_index.llms.gemini import Gemini
from pydantic import BaseModel
from typing import Dict, Union
import json






# Response Schema
class Response(BaseModel):
    response: str
    status: str  # "normal", "ordering", or "done"
llm = Gemini(
    model="models/gemini-1.5-flash",
)
structured_llm = llm.as_structured_llm(output_cls=Response)

/Users/hamza.mahmood/Developer/ezrai/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
import sys
import re


class Item(BaseModel):
    ordered: bool
    name: str
    quantity: int

def preprocess_and_parse(input_string):
    # Remove markdown code block indicators if present
    # json_string = re.sub(r'^```json\s*|\s*```$', '', input_string.strip())
    json_pattern = r'\{(?:[^{}]|\{[^{}]*\})*\}|\[(?:[^\[\]]|\[[^\[\]]*\])*\]'
    json_match = re.search(json_pattern, input_string)
    
    if not json_match:
        return None
    
    json_str = json_match.group(0)
    
    # Use regex to find the "value" field and process its numeric value
    processed_string = re.sub(
        r'"quantity"\s*:\s*"?(\d+(?:,\d+)*(?:\.\d+)?)"?', 
        lambda m: f'"quantity": {m.group(1).replace(",", "")}',
        json_str
    )
       
    # Parse the processed string as JSON
    print(f"processed string: \n{processed_string}")
    return json.loads(processed_string)

# Chat Handler to manage conversation history and process query
class ChatHandler:
    def __init__(self, llm):
        self.history = []
        self.current_status = "normal"  # Initially in "normal" status
        self.order_items = {}  # Track items separately
        self.llm = llm

    def add_to_history(self, user_input, ai_response):
        """Store conversation history."""
        self.history.append(f"User: {user_input}")
        self.history.append(f"AI: {ai_response}")

        # Limit history to avoid excessive token use
        if len(self.history) > 10:
            self.history = self.history[-10:]

    def process_query(self, query) -> Response:
        """Forces every response into Response format while keeping chat history."""
        history_str = "\n".join(self.history)
        prompt = f"""
        {SYSTEM_PROMPT}

        Chat History:
        {history_str}

        Current Status: {self.current_status}

        User: {query}
        
        Respond according to schema: {Response.model_json_schema()}
        ordering if the intent of customer is to order, done if they are done ordering, normal for all other queries
        """
        
        
        raw_response = self.llm.complete(prompt).text.strip()

        try:
            response_data = json.loads(raw_response)
            # Ensure status is always present, fallback to "normal"
            status = response_data.get("status", "normal")
            response_data["status"] = status  # Set status if missing
            self.current_status = status  # Update status tracking
            self.add_to_history(query, response_data["response"])
            return Response(**response_data)
        except json.JSONDecodeError:
            return Response(response="I'm not sure how to respond.", status="normal")


# Order Management (Cart & Checkout)
class OrderModule:
    def __init__(self):
        self.cart = {}

    def add_to_cart(self, item, quantity=1):
        """Adds items to the cart."""
        if item in self.cart:
            self.cart[item] += quantity
        else:
            self.cart[item] = quantity
        return f"Added {quantity}x {item} to your cart."

    def view_cart(self):
        """View the current cart."""
        if not self.cart:
            return "Your cart is empty."
        return "\n".join([f"{item}: {quantity}" for item, quantity in self.cart.items()])
    
    def return_json(self):
        return [{"name": k, "quantity": v} for k,v in self.cart.items()]

    def confirm_order(self):
        """Finalizes the order and returns all ordered items in JSON format."""
        if not self.cart:
            return Response(response="No items to order.", status="done", order_items={})

        order_summary = self.cart.copy()
        self.cart.clear()

        return Response(
            response="Order placed successfully! Is there anything else I can help you with?",
            status="done",
            order_items=order_summary
        )


class OrderingChatHandler:
    def __init__(self, llm) -> None:
        self.system_prompt = ""
        self.convo_starter = "What would you like to order"
        self.ask_more = "Can I help you with anything else?"
        self.ask_instruction = "Do you have any instructions?"
        self.order_str = ""
        self.confirm_order = f"Here is summary of your order: {self.order_str}"
        self.order = {}
        self.llm = llm
        self.history = []
        self.status = 0
        self.prev_response = None
    
    class Item(BaseModel):
        name: str
        quantity: int
        
    def process_query(self,query):
        history_str = "\n".join(self.history)
        prompt = f"""
                {SYSTEM_PROMPT}

                Chat History:
                {history_str}

                Customer is trying to order something. Extract the order items from this customer response.

                Follow JSON schema: {[Item.model_json_schema()]}
                Ordered = 1 if the user ordered anything, 0 if not. Leave name "NA" and quantity 0 if the user did not order.
                
                User: {query}
                """
        raw_response = llm.complete(prompt).text.strip()
        try:
            response_data = preprocess_and_parse(raw_response)
            self.prev_response = response_data
            if isinstance(response_data, list):
                if int(response_data[0]['ordered']) == 1:
                    self.status = 1
                else:
                    self.status = 0
            else:
                response_data = [response_data]
            return response_data
        except:
            return {"ordered": 0, "name": "NA", "quantity": 0}
    
    
        

# Main conversation loop
def conversation_loop():
    order_system = OrderModule()
    chat_handler = ChatHandler(llm=structured_llm)
    order_chat_handler = OrderingChatHandler(llm=llm)

    while True:
        user_input = input("Customer: ")
        if user_input.lower() in ["exit", "quit"]:
            print("Goodbye!")
            break

        classification = chat_handler.process_query(user_input)
        print(f"AI: {classification}")
        
            
        if classification.status == "ordering":
            # extract items using llm
            print("ordering.....")
            while order_chat_handler.process_query(user_input)[0]['ordered']:
            # items_ordered = user_input.lower().split(" and ")  # Handling multiple items in one query
                user_order = order_chat_handler.prev_response
                for item in user_order:
                    item_name = item["name"]
                    quantity = item["quantity"]
                    order_system.add_to_cart(item_name, quantity=quantity)
                print(f"--------\n{order_system.view_cart()}\n--------")
                user_input = input("do you need anything else?")

            print("--------Order done------")
            print("#### CART #####")
            print(order_system.return_json())
            push_order(order=order_system.return_json())
            ########
            
            
            # Confirm order
            
            #########
            # user_order = order_chat_handler.process_query(user_order)
            # print(user_order)
            # sys.exit(2)
            # while True:
            #     # Asking if the customer wants anything else
            #     user_query = input("Customer (ordering): ")
            #     classification = order_chat_handler.process_query(user_query)
            #     print(classification.status)
            #     if classification.status == "done":
            #     #if user_query.lower() in ["done", "checkout", "finish"]:
            #         order_response = order_system.confirm_order()
            #         print(f"AI: {order_response.response}")
            #         print(f"Final Order: {json.dumps(order_response.order_items, indent=2)}")

            #         # Ask for the customer name and phone number
            #         print("AI: What is your name?")
            #         name = input("What is your name? ")
            #         print("AI: What is your Phone number?")
            #         phone_number = input("Can I have your phone number for the order? ")

            #         # Store customer information
            #         print(f"Thank you, {name}. Your order has been placed. We will contact you at {phone_number} if necessary.")
            #         break
            #     item_classification = chat_handler.process_query(user_query)

            #     if item_classification.status != "ordering":
            #         print("taking order")
            #         print(f"AI: {item_classification.response}")
                    
            #     else:
            #         r = order_system.add_to_cart(user_query, quantity=1)
            #         print(f"added to cart: {r}")
            #         print("Cart:", order_system.view_cart())


### NOTES
Pass in structured llm for normal convo like before. Not for ordering. 

In [9]:
conversation_loop()

AI: response='Okay, one slider and one order of loaded fries. Will there be anything else?' status='ordering'
ordering.....
processed string: 
[
  {
    "ordered": 1,
    "name": "Slider",
    "quantity": 1
  },
  {
    "ordered": 1,
    "name": "Loaded Fries",
    "quantity": 1
  }
]
--------
Slider: 1
Loaded Fries: 1
--------
processed string: 
[
  {
    "ordered": 0,
    "name": "NA",
    "quantity": 0
  }
]
--------Order done------
#### CART #####
[{'name': 'Slider', 'quantity': 1}, {'name': 'Loaded Fries', 'quantity': 1}]
✅ Order [{'name': 'Slider', 'quantity': 1}, {'name': 'Loaded Fries', 'quantity': 1}] added.
AI: response='Okay, one slider and one order of loaded fries. That will be $20.98. Will there be anything else?' status='ordering'
ordering.....
processed string: 
[]


KeyError: 0